In [1]:
# pip install tweepy

Note: you may need to restart the kernel to use updated packages.


### Data Extraction from Twitter API

In [10]:
import tweepy
import json
import time
import datetime
from datetime import date, timedelta

import pandas as pd
import re
import string
import seaborn as sns

In [6]:
api_key = '1D9l2cpV4EJaO1jBrdWU9zfoU' 
api_key_secret = 'qxyDTwZUABqd0sjaLe1Q3wQSxBhTjRexWbxDptu34fWt51ohpI'
access_token = '1572868505137188864-JSJF2tFizDiSvvMTUH5UhGjd1wOn5j'
access_token_secret = '6A562B7011LSkzmQs4TNZMLPO3USIw7mh5p89LlyKhOfD'

In [7]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
#api.search_tweets()

In [37]:
list_of_dicts = []
search_term = 'covid19 covid vaccine'
num_tweets = 20000

In [39]:
list_of_dates = []
today = date.today()
for i in range(-7,1):
    target_date = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    list_of_dates.append(target_date)

In [42]:
def get_tweets(search_term = search_term, num_tweets = num_tweets):
#     try:
        for end_date in list_of_dates:
            start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d") # Create 1-day windows for extraction
            tweet_count = len(list_of_dicts)

            for tweet in tweepy.Cursor(api.search_tweets,
                                       q=f'{search_term} since:{start_date} until:{end_date}',
                                      # q=search_term,
                                       lang = 'en',
                                       count = num_tweets,
                                       tweet_mode = 'extended').items(num_tweets):
                #if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if 'RT @' not in tweet.full_text:
                    if tweet.lang == "en":
                        tweet_dict = {}
                        tweet_dict['username'] = tweet.user.name
                        tweet_dict['location'] = tweet.user.location
                        tweet_dict['text'] = tweet.full_text 
                        tweet_dict['hashtags'] = tweet.entities['hashtags']
                        dtx = tweet.created_at
                        tweet_dict['tweet_date'] = dtx.strftime("%m/%d/%Y, %H:%M:%S")
                        tweet_dict["retweeted"] = tweet.retweeted
                        tweet_dict["retweet_count"] =  tweet.retweet_count
                        tweet_dict["favorite_count"] = tweet.favorite_count
                        list_of_dicts.append(tweet_dict)
                        tweet_count +=1
                        #if tweet_count % 20 == 0:
                            #print(f'Extracted tweet count = {tweet_count}')
                        
                        #print(tweet_count)
                        
            #print(f'Completed extraction for {start_date} to {end_date}. Sleep for 15 mins')
            time.sleep(900)
            #print('Ready again')

In [45]:
get_tweets()

In [51]:
# TO BE UNCOMMENTED FOR DATA EXTRACTION TESTING
df = pd.DataFrame(list_of_dicts)
len(df)
# ============
# s_set = {}

# for idx in range(0, len(df)):
#     s = df["tweet_date"][idx].split(",")[0]
#     #print(type(s))
#     if s in s_set.keys():
#         s_set[s] += 1
#     else:
#         s_set[s] = 1
    
# s_set
#=============
df.to_csv("../data/vaccination_all_tweets.csv")

206967

In [49]:
df = pd.read_csv("../data/vaccination_all_tweets.csv")